In [1]:
import pickle
import pandas as pd
import numpy as np

from pathlib import Path
from sklearn.pipeline import Pipeline

In [19]:
path_src_dataset = "/home/jovyan/hfactory_magic_folders/water_shortage_prediction/X_test_Hi5.csv"

path_folder_config = "./data/processed/pipelines"
pipeline_name = "pip_yael_1h45"

path_submission = Path("./data/submissions")
path_submission.mkdir(parents=True, exist_ok=True)

target = "piezo_groundwater_level_category"

In [10]:
col_yass = ['meteo_date'] + ['prelev_volume_0', 'prelev_volume_1',
                             'prelev_volume_2', 'prelev_other_volume_sum', 'piezo_station_commune_name']
# Altitude
altitude_flo = ["piezo_station_altitude", "meteo_altitude"]  # ORDRE 1
prelev_flo = ["prelev_volume_0", "prelev_usage_label_0", "prelev_volume_obtention_mode_label_0", "prelev_volume_1", "prelev_usage_label_1",
              "prelev_volume_obtention_mode_label_1", "prelev_volume_2", "prelev_usage_label_2", "prelev_volume_obtention_mode_label_2"]
col_flo = altitude_flo + prelev_flo
# Insee & rain "CleanFeatures"
cols_yael_input = ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune',
                   'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
cols_yael_need = ["piezo_station_department_code", "meteo_date"]

# Temperature
cols_lucien_need = ['piezo_station_department_code', 'piezo_measurement_date']
cols_lucien_input = ['meteo_temperature_avg', 'meteo_temperature_min',
                     'meteo__pressure_saturation_avg', 'meteo_temperature_max']
# Lat Long
cols_mat = ["distance_piezo_meteo", 'piezo_station_longitude', 'piezo_station_latitude', 'meteo_latitude', 'meteo_longitude', "meteo_temperature_avg", "meteo_temperature_avg_threshold",
            "meteo_temperature_min", "meteo_temperature_max", "meteo_temperature_min_ground", "hydro_observation_result_elab", "hydro_status_code", "hydro_qualification_code", "hydro_hydro_quantity_elab"]


# Clean pizo
pizo_cols = ['piezo_station_investigation_depth', 'piezo_obtention_mode', 'piezo_status',
             'piezo_qualification', 'piezo_measure_nature_code', 'piezo_station_department_code']

# target
target = "piezo_groundwater_level_category"


columns_to_keep = col_yass + cols_yael_input + cols_yael_need + col_flo + \
    cols_lucien_need + cols_lucien_input + cols_mat + pizo_cols + ["row_index"]

In [11]:
df = pd.read_csv(path_src_dataset, usecols=columns_to_keep, dtype={"row_index": int})

/tmp/ipykernel_16015/9862667.py:2: DtypeWarning: Columns (1,110,111,117,118,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_src_dataset, usecols=columns_to_keep, dtype={"row_index": int})


In [12]:
with open(path_folder_config / Path(pipeline_name + ".pkl"), 'rb') as file:
    pipeline: Pipeline = pickle.load(file)

In [14]:
df_test_pred = pipeline.predict(df.drop(columns="row_index"))

>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_2' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_2' has been one-hot-encoded in 4 features
>> (Info) Filling missing values with calculated medians and means.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means or global mean as fallback.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding app

In [15]:
df_test_pred = pd.concat([df["row_index"], pd.DataFrame(df_test_pred, columns=[target])], axis=1)

mapping = {0: 'Very Low', 1: 'Low', 2: 'Average', 3: 'High', 4: 'Very High'}
df_test_pred[target] = df_test_pred[target].map(mapping)

In [16]:
df_test_pred.shape

(611208, 2)

In [20]:
df_test_pred.to_csv(Path(path_submission) / Path(pipeline_name + "_group43_yael" + ".csv"), index=False)